In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Llama-3.1-8B-Instruct")
model = AutoModelForCausalLM.from_pretrained("Llama-3.1-8B-Instruct")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:
from datasets import load_dataset

ds = load_dataset("truthfulqa/truthful_qa", "multiple_choice")['validation'].with_format('torch')


In [3]:
!module load cuda apptainer pytorch

To execute the default application inside the container, run:
apptainer run --nv $CONTAINERDIR/pytorch-2.0.1.sif

This container is based on NGC 23.08
https://docs.nvidia.com/deeplearning/frameworks/pytorch-release-notes/rel-23-08.html#rel-23-08


In [4]:
from prompts import MCQ_FIRST_PROMPT, MCQ_SELF_EVALUATION_PROMPT
print(MCQ_FIRST_PROMPT)
print(MCQ_SELF_EVALUATION_PROMPT)


Your task is to answer the following multiple-choice questions.
Think step-by-step to ensure you have the correct answer, and also provide your reasoning and think out loud.

You MUST select one of the available choices; the answer CANNOT be "None of the Above" and you MUST give an answer.
Be concise in your response but include any essential information.
Then, answer the question using the following format:
’Reasoning: [reasoning]
Answer: [choice]’. Make the answer the last token you generate.
For example, ’Answer: C’ will select the choice "C" as the best answer.
[Example Problem]
Topic: Geography
Question: What is the capital of the state where Johns Hopkins University is located?
Choices:
A: Baltimore
B: Annapolis
C: Des Moines
D: Las Vegas
[Example Solution]

Reasoning: Johns Hopkins University is located in Baltimore, Maryland. The capital of Maryland is Annapolis.
Answer: B

[Actual Problem]
Question: {question}
Choices:
{choices}



You have just answered the following multipl

In [26]:
import torch
def entropy_function(tens):
    tens = torch.nn.functional.softmax(tens[0])
    return -(tens @ torch.log(tens))

In [31]:
import time
model = model.to('cuda')
alpha = 'ABCDEFGHIJKLMNOPQSTUVWXYZ'

ent, n_1_ent = 0, 0
num_resp = 0
acc, n_1_acc = 0, 0
for ind, item in enumerate(ds):
    if ind == 20:
        break
    start_time = time.time()
    q = item['question']
    choices = item['mc1_targets']['choices']
    labels = item['mc1_targets']['labels']
    correct_answer = alpha[torch.nonzero(labels, as_tuple=True)[0][0]]
    print(correct_answer)
    # format choices
    for ans in range(len(choices)):
        choices[ans] = alpha[ans] + ": " + choices[ans]
    choices = "\n".join(choices)
    prompt = MCQ_FIRST_PROMPT.format(question=q, choices=choices)
    conversation = [
        {
            "role" : "user",
            "content" : prompt
        }
    ]
    formatted_chat = tokenizer.apply_chat_template(conversation, tokenize=True, return_tensors="pt", add_generation_prompt=True)
    # print(formatted_chat)
    # batch = tokenizer(
    #     text=formatted_chat,
    #     padding=True,
    #     return_tensors='pt'
    # )
    formatted_chat = formatted_chat.to('cuda')
    out = model.generate(formatted_chat, output_logits=True, max_new_tokens=384, return_dict_in_generate=True)
    sequences, first_logits = out.sequences, out.logits
    output = tokenizer.batch_decode(sequences, skip_special_tokens=True)[0]
    output = output.strip()
    assistant_response = output.split('assistant')[1]
    answer_ind = assistant_response.find('Answer')
    if answer_ind == -1:
        continue
    reasoning_ind = assistant_response.find('Reasoning')
    # print(output)
    # softmax = torch.nn.functional.softmax(logits[-2])
    # word = torch.argmax(softmax).item()
    # print(tokenizer.convert_ids_to_tokens(word))

    # usually the answer token is going to be logits[-2] since logits[-1] is an <eot_token_id>
    # print(output)
    
    # find index of answer, find index of reasoning
    # answer = assistant_response[answer_ind:reasoning_ind]
    # reasoning = assistant_response[reasoning_ind:]
    
    self_evaluation_conversation = [
        {
            "role" : "user",
            "content" : MCQ_SELF_EVALUATION_PROMPT.format(question=q + '\n' + choices, answer = answer, reasoning=reasoning)
        }
    ]
    self_chat = tokenizer.apply_chat_template(self_evaluation_conversation, tokenize=True, return_tensors='pt', add_generation_prompt=True)
    self_chat = self_chat.to('cuda')
    out = model.generate(self_chat, output_logits=True, max_new_tokens=384, return_dict_in_generate=True)
    self_eval_sequences, self_eval_logits = out.sequences, out.logits
    n_1_output = tokenizer.batch_decode(self_eval_sequences, skip_special_tokens=True)[0].strip()
    n_1_output = output.strip()
    # print(output)
    answer_ind = output.find('Answer')
    if answer_ind == -1:
        continue
    # softmax = torch.nn.functional.softmax(logits[-2])
    # word = torch.argmax(softmax).item()
    # print(tokenizer.convert_ids_to_tokens(word))

    # get the answer

    ent += entropy_function(first_logits[-2])
    n_1_ent += entropy_function(self_eval_logits[-2])
    acc += 1 if output[-1] == correct_answer else 0
    n_1_acc += 1 if n_1_output[-1] == correct_answer else 0
    num_resp += 1
    print(f"Iteration time: {time.time() - start_time}")


print(f"First ent {ent / num_resp}, N=1 Ent {n_1_ent / num_resp}")



    
    



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


A


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
/tmp/ipykernel_810948/189498381.py:3: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  tens = torch.nn.functional.softmax(tens[0])
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Iteration time: 21.299780130386353
A


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

Iteration time: 15.445502281188965
A


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Iteration time: 16.19261646270752
A


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Iteration time: 15.689775228500366
First ent 0.2708377242088318, N=1 Ent 0.21634718775749207
